# 3. Preparación de los datos

## 3.1 División de los datos en conjuntos de datos de entrenamiento, validación y prueba

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/df_patients_observations_medications_INR.csv")


In [2]:
# copy the original dataframe so further manipulations will not affect it 
df_select = df.copy() 
categorical = df_select.select_dtypes(include=['object']).columns.tolist()
numerical = df_select.select_dtypes(include=['int64','float64']).columns.tolist()
numerical.remove('INR')

In [3]:
from sklearn.model_selection import train_test_split

# separate train + validation (= full) and test
df_full_train, df_test = train_test_split(df_select, test_size=0.2, random_state=1)
# now split the full into train and val, it should be the 20% of the 80%, which is 20/80=1/4=0.25
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1) 
len(df_train), len(df_val), len(df_test)

(325, 109, 109)

In [4]:
# reset index
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

# separate the target
y_train = df_train.INR.values
y_val = df_val.INR.values

# remove the target from the features
del df_train['INR']
del df_val['INR']

## 3.2 Codificación de las variables categóricas

Usamos Scikit-Learn DictVectorizer para codificar características categóricas (toma un diccionario y lo convierte en un vector, es decir, en un numpy.array). Es un método de los llamados One-Hot Encoding (OHE) para convertir las características categóricas (no afectaría las numéricas) en columnas con valores binarios, con tantas columnas como valores tome la variable categórica. Como veremos a continuación, esta codificación proporciona un buen rendimiento del modelo de referencia inicial ('baseline model'), por lo que no exploramos los efectos de otros algoritmos de codificación, como la función get_dummies() o la librería Category Encoders, por ejemplo.


In [5]:
from sklearn.feature_extraction import DictVectorizer 

dv = DictVectorizer(sparse=False) # False bcs is not a sparse matrix (we do not have many zeros)

# TRAIN
train_dict = df_train[categorical].to_dict(orient='records') # records = to do it row-wise, not col-wise
X_train_cat = dv.fit_transform(train_dict) # make it a vector

# VAL
val_dict = df_val[categorical].to_dict(orient='records')
X_val_cat = dv.transform(val_dict)

Ahora bien, nuestro dataframe de las 'features' categóricas tiene información redundante. Por ejemplo, la columna de la caracteristíca 'ExcerciseAngina' que podía tener valores 'N' o 'Y' ahora dió lugar a dos columnas, una llamada 'ExcerciseAngina=N' y otra 'ExcerciseAngina=Y' que claramente no son independientes y están totalmente correlacionadas, miremos por ejemplo, el conjunto de entrenamiento:

**LAS CORRELACIONES SON LAS MISMAS QUE ANTERIORMENTE**

In [ ]:
# convert de data array to a dataframe
df_X_train_cat = pd.DataFrame(X_train_cat,columns= dv.get_feature_names_out())
X_train_cat = df_X_train_cat.to_numpy()

Hacemos lo mismo con el conjunto de validación:

In [ ]:
# convert the data array to a dataframe
df_X_val_cat = pd.DataFrame(X_val_cat,columns= dv.get_feature_names_out())

# remove redundant columns
df_X_val_cat.drop(['ExerciseAngina=N', 'FastingBS=H', 'Sex=F', 'ST_Slope=Up'], 
                    axis=1, inplace=True)

# convert the dataframe to a np.array again
X_val_cat = df_X_val_cat.to_numpy()

y los nombres de las columnas con 'features' categoricas que ha sido codificadas son:

In [ ]:
categorical_update = df_X_train_cat.columns.to_list()

## 3.3 Scaling de las variables numéricas

Usamos Scikit-Learn StandardScaler para acotar cada característica numérica sustrayendo su valor medio y dividiendo el resultado entre su desviación estándar de manera que al final los valores de esa columna se encuentren entre 0 y 1. Si no aplicamos esta acotación, las columnas con valores en un rango más alto tendrían más representación y varios modelos como la regresión logística no convergen. Otros modelos, como los basados en árboles, no son sensibles a la diferencia entre los rangos de las características y no es un necesario aplicar este procedimiento. En inglés este procedimiento se conoce como "scaling", y a veces se traduce como "normalización" porque al final todas las características siguen la misma "norma" o "rango", pero como tantas otras veces, aquí los nombres pueden dar lugar a confusión (ver otra definición de "normalización" por ejemplo en esta referencia, donde el resultado final es una distribución gaussiana para cada columna). Como veremos a continuación, acotar los datos proporciona un rendimiento razonable para el modelo de referencia inicial, por lo tanto, no exploramos los efectos de otros algoritmos de estandarización, como MixMaxScaler, por ejemplo).

In [ ]:
from sklearn.preprocessing import StandardScaler 

scaler = StandardScaler()

# TRAIN
X_train_num = df_train[numerical].values
X_train_num = scaler.fit_transform(X_train_num)

# VAL
X_val_num = df_val[numerical].values
X_val_num = scaler.transform(X_val_num)

Unimos las matrices numéricas y categóricas:

In [ ]:
# TRAIN
X_train = np.column_stack([X_train_num, X_train_cat])

# VAL
X_val = np.column_stack([X_val_num, X_val_cat])

# 4. Entrenamiento de los modelos

En esta división aleatoria, y en todas las ocasiones siguientes en las que usemos un generador de números aleatorios, fijamos la semilla del generador de números aleatorios para asegurar la reproducibilidad de los resultados. Fijamos el random_state = 1 pero podría ser 2 o cualquier otro valor, no afectará significativamente a las prediciones, pero con esta corrección siempre obtendremos exactamente los mismos resultados.

¡Perfecto! Te puedo dar un ejemplo básico en Python usando **scikit-learn**, **XGBoost** y **CatBoost** para una variable continua, listo para usar en un notebook `.ipynb`. Supondré que tienes un dataset `X` (variables independientes) y `y` (variable continua).

---

```python
# Instalar librerías si no las tienes
!pip install scikit-learn xgboost catboost --quiet

# Importar librerías
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Modelos
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# Simulamos un dataset de ejemplo
from sklearn.datasets import make_regression
X, y = make_regression(n_samples=500, n_features=10, noise=0.2, random_state=42)

# Dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
```

---

### 1️⃣ **Decision Tree Regressor**

```python
# Crear modelo
dt_model = DecisionTreeRegressor(random_state=42)
# Entrenar
dt_model.fit(X_train, y_train)
# Predecir
y_pred_dt = dt_model.predict(X_test)
# Evaluar
print("Decision Tree R2:", r2_score(y_test, y_pred_dt))
print("Decision Tree RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_dt)))
```

---

### 2️⃣ **Random Forest Regressor**

```python
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

print("Random Forest R2:", r2_score(y_test, y_pred_rf))
print("Random Forest RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_rf)))
```

---

### 3️⃣ **XGBoost Regressor**

```python
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

print("XGBoost R2:", r2_score(y_test, y_pred_xgb))
print("XGBoost RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_xgb)))
```

---

### 4️⃣ **CatBoost Regressor**

```python
# silent=True evita que imprima el progreso del entrenamiento
cat_model = CatBoostRegressor(n_estimators=100, learning_rate=0.1, random_state=42, silent=True)
cat_model.fit(X_train, y_train)
y_pred_cat = cat_model.predict(X_test)

print("CatBoost R2:", r2_score(y_test, y_pred_cat))
print("CatBoost RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_cat)))
```

---

💡 **Notas importantes:**

* Ajusta los hiperparámetros (`n_estimators`, `learning_rate`, `max_depth`) según tu dataset.
* Para **interpretabilidad**, los árboles de decisión y Random Forest permiten usar `feature_importances_`.
* XGBoost y CatBoost suelen **dar mejor desempeño** en datasets más complejos.

---

Si quieres, puedo hacer una **versión resumida de este código** que entrene los **3 modelos recomendados** y los compare automáticamente en un solo bloque, lista para copiar y pegar en Jupyter. Esto hace todo más limpio y rápido.

¿Quieres que haga eso?
